###### Reading the required libraries

In [158]:
import pandas as pd
import glob
import os
import numpy as np

###### Reading the required data files and combining them to a single dataframe

In [3]:
path = r'D:\Power-Bi\Google-Data-Analytics-Capstone-Case-Study\Data' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))
##D:\Power Bi\Google-Data-Analytics-Capstone-Case-Study\Data
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
frame = pd.concat(li, axis=0, ignore_index=True)

###### Preparing and processing the data for analysis

In [152]:
## Taking only the required columns
frame_req = frame[['ride_id','rideable_type','started_at','ended_at','start_station_id','end_station_id','member_casual']]

## Converting the datatypes to required ones
frame_req['started_at'] =  pd.to_datetime(frame_req['started_at'])
frame_req['ended_at'] =  pd.to_datetime(frame_req['ended_at'])

## Calculating the ride time based on stand and end time
frame_req['ride_time'] =  frame_req['ended_at'] - frame_req['started_at']

C:\Users\email\AppData\Local\Temp/ipykernel_3004/3292728272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_req['started_at'] =  pd.to_datetime(frame_req['started_at'])
C:\Users\email\AppData\Local\Temp/ipykernel_3004/3292728272.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_req['ended_at'] =  pd.to_datetime(frame_req['ended_at'])
C:\Users\email\AppData\Local\Temp/ipykernel_3004/3292728272.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [153]:
## Checking number of null values in each columns
frame_req.isna().sum()

## Number of null values:
## start_station_id    833064
## end_station_id      892742

## Dropping all rows with null values, woth the assumption that there is no start or end station id, 
## which could be error or corrupt records for analysis
frame_req = frame_req.dropna(how='any',axis=0) 

## Re-chekcing for missing values
frame_req.isna().sum() # 0

ride_id             0
rideable_type       0
started_at          0
ended_at            0
start_station_id    0
end_station_id      0
member_casual       0
ride_time           0
dtype: int64

In [154]:
## Less than 2% entries are below 5mins in which few are in minus ride times
## and around 3% entries are more than 1 hour, in which few are for many days
## For the analysis and finding trend purpose purpose these records have been removed
frame_req = frame_req[frame_req['ride_time'] > '00:05:00']
frame_req = frame_req[frame_req['ride_time'] < '01:00:00'] 

In [155]:
## Checking for the type of riders against their ride type
frame_req[['ride_id','rideable_type','member_casual']].groupby(['rideable_type','member_casual']).agg(['count'])

## Docked bike is used only by casual users, thus comparision is not possible nor 
## there is any information about what docked bike is, thus removed from analysis
frame_req = frame_req[frame_req['rideable_type'] != 'docked_bike']

In [156]:
## Finding the data of the week when a ride starts to find patterns
frame_req['day_of_week'] = frame_req['started_at'].dt.day_name()

In [159]:
conditions = [
    (frame_req['ride_time'] > '00:00:00') & (frame_req['ride_time'] <= '00:10:00'),
    (frame_req['ride_time'] > '00:10:00') & (frame_req['ride_time'] <= '00:20:00'),
    (frame_req['ride_time'] > '00:20:00') & (frame_req['ride_time'] <= '00:30:00'),
    (frame_req['ride_time'] > '00:30:00') & (frame_req['ride_time'] <= '00:40:00'),
    (frame_req['ride_time'] > '00:40:00') & (frame_req['ride_time'] <= '00:50:00'),
    (frame_req['ride_time'] > '00:50:00') & (frame_req['ride_time'] <= '00:60:00'),
]

values = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60']

frame_req['Range'] = np.select(conditions, values)

In [164]:
ride_counts = frame_req[['ride_id','rideable_type','member_casual']].groupby(['rideable_type','member_casual']).agg(['count'])
ride_counts = ride_counts.reset_index()

In [168]:
frame_req[['ride_id','rideable_type','Range']].groupby(['Range','rideable_type']).agg(['count'])


ride_id
                      count
Range rideable_type        
0-10  classic_bike   731734
      electric_bike  513321
10-20 classic_bike   772209
      electric_bike  478110
20-30 classic_bike   308125
      electric_bike  147284
30-40 classic_bike   144539
      electric_bike   55427
40-50 classic_bike    67150
      electric_bike   26392
50-60 classic_bike    31469
      electric_bike   14605